In [2]:
import json
import arrow
from building_depot import DataService
import pandas as pd

In [5]:
with open("config/ion_config.json", "r") as fp:
    configs = json.load(fp)
    base_url = configs['base_url']
    start_time = arrow.get(configs['start_time']).datetime
    end_time = arrow.get(configs['end_time']).datetime
with open("config/bd2_3credential.json", "r") as fp:
    cred = json.load(fp)
    apikey = cred['api_key']
    username = cred['username']
    
datadir = "./data/"

In [6]:
def make_series(bd_data):
    time_list = list()
    value_list = list()
    for datum in data:
        time_list.append(datum.keys()[0])
        value_list.append(datum.values()[0])
    return pd.Series(index=time_list, data=value_list)

In [7]:
bd = DataService(base_url, apikey, username)
orig_metadata_df = pd.DataFrame.from_csv('metadata/ion.txt', )

In [8]:
srcid_dict = dict()
for uuid in orig_metadata_df['uuid']:
    data = bd.get_timeseries_datapoints(uuid, "PresentValue", start_time, end_time)['timeseries']
    if len(data)>1:
        sensor = bd.view_sensor(uuid)
        srcid_dict[uuid] = sensor['source_identifier']
        make_series(data).to_csv(datadir+uuid+'.csv')

IOError: [Errno 2] No such file or directory: './data/001dfb32-038a-4559-9064-f4586c8fefb0.csv'